In [3]:
import pandas as pd
from io import StringIO
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import ezkl
import torch
import torch.nn as nn
import torch.optim as optim
import os
from scipy import stats
import scipy
import onnxruntime
from dataclasses import dataclass
from typing import List, Dict


# Monte Carlo Simulations


## Defining LTV

LTV refers to Loan To Value

LTV can be defined as follows

l = loan value (we assume this to be usd)

c = collateral value

so,

$$ LTV = \frac{l}{c}$$

A safe LTV would be equivalent to $1 - σ$ where $\sigma$ stands for standard deviation and add some buffer `ϵ` just in case. we multiply by $\sqrt{252}$ to get the annualized volatility.

$$ SafeLTV = \frac{l}{c} = 1 - σ * \sqrt{252}$$


## Positions
A position consists of

1. collateral_amount
2. borrowed_amount
3. collateral_asset
4. debt_asset


In [4]:
@dataclass
class Position:
    collateral_amount: float
    collateral_asset: str
    borrowed_amount: float
    borrowed_asset: str


## Sentiment Vault

This is a simplified version. To make this more detailed so we get a better representation of the actual vault

Do install the ezkl binary via 
```
curl https://raw.githubusercontent.com/zkonduit/ezkl/main/install_ezkl_cli.sh | bash
```

In [23]:
import subprocess

def run_ezkl_gen_witness(
    data="input.json", 
    model="model.compiled", 
    output="witness.json"
):
    """
    Run ezkl gen-witness command using subprocess
    
    Args:
        data_path (str): Path to input JSON data file
        circuit_path (str): Path to compiled circuit file
        output_path (str): Path for output witness file
    
    Returns:
        subprocess.CompletedProcess: Result of the subprocess call
    """
    cmd = [
        "ezkl", "gen-witness",
        "-D", data,
        "-M", model, 
        "-O", output
    ]
    
    return subprocess.run(cmd, 
                        check=True,
                        capture_output=True,
                        text=True)


In [24]:
from collections import deque
import time

class DeFiVault:
    def __init__(self, initial_liquidity, assets=['usdc', 'eth']):
        # TODO: change liquidity based on eth price
        self.initial_liquidity = initial_liquidity
        self.total_liquidity = initial_liquidity
        self.positions: List[Position] = []
        self.liquidation_history = []
        self.negative_total_liquidity_history = []
        self.assets = assets
        self.price_history = {}

    def initialize_price_history(self, asset: str, initial_prices: List[float]):
        """Initialize the price history deque for an asset"""
        if len(initial_prices) != 20:
            raise ValueError(f"Must provide exactly 20 initial prices for {asset}")
        self.price_history[asset] = deque(initial_prices, maxlen=20)

    def update_price_history(self, asset: str, new_price: float):
        """Add a new price to the rolling window"""
        self.price_history[asset].append(new_price)

    def get_ltv(self) -> float:
        """Calculate LTV using EZKL GARCH model"""
        if len(self.price_history['eth']) != 20:
            raise ValueError(f"Need 20 price points for eth")

        # Prepare input for EZKL
        with open(os.path.join("data", "input_sim.json"), "w") as f:
            json.dump({"input_data": [list(self.price_history['eth'])]}, f)

        # Generate witness using EZKL

        try:
            res = run_ezkl_gen_witness(
                data=os.path.join("data", "input_sim.json"),
                model=os.path.join("data", "model.compiled"),
                output=os.path.join("data", "witness_sim.json")
            )
        except subprocess.CalledProcessError as e:
            print(f"Error: {e.stderr}")
            
        # add a sleep to 
        time.sleep(1)

        if res:
            # Read results
            with open(os.path.join("data", "witness_sim.json"), "r") as f:
                data = json.load(f)

            value = float(data['pretty_elements']['rescaled_outputs'][0][0])

            return 1 - value

    def add_position(
            self,
            position: Position,
        ):
        """Add a new lending/borrowing position"""
        self.positions.append(position)

        collateral_value = (position.collateral_amount *
                            self.price_history[position.collateral_asset][-1])

        borrowed_value = (position.borrowed_amount *
                          self.price_history[position.borrowed_asset][-1])

        self.total_liquidity += collateral_value
        self.total_liquidity -= borrowed_value

    def check_liquidations(
        self,
        timestamp: int,
    ) -> List[Position]:
        """Check and process liquidations based on current prices and LTV"""
        liquidated = []
        current_ltv = self.get_ltv()

        for position in self.positions:
            collateral_value = (
                position.collateral_amount *
                self.price_history[position.collateral_asset][-1]
            )
            borrowed_value = (
                position.borrowed_amount *
                self.price_history[position.borrowed_asset][-1]
            )

            # Calculate current LTV ratio
            position_ltv = borrowed_value / collateral_value

            if position_ltv > current_ltv:
                self.liquidate_position(position, self.price_history['eth'][-1])
                liquidated.append(position)
                self.liquidation_history.append({
                    'timestamp': timestamp,
                    'collateral_amount': position.collateral_amount,
                    'borrowed_amount': position.borrowed_amount,
                    'ltv_at_liquidation': position_ltv
                })

            if self.total_liquidity < self.initial_liquidity:
                self.negative_total_liquidity_history.append({
                    'timestamp': timestamp,
                    'total_liquidity': self.total_liquidity
                })

        # Remove liquidated positions
        self.positions = [p for p in self.positions if p not in liquidated]
        return liquidated

    def liquidate_position(self, position: Position, current_price):
        """Process a liquidation"""
        collateral_value = (position.collateral_amount *
                          current_price)
        self.total_liquidity += collateral_value

## Simulate Paths
We simulate the price paths using Geometric Brownian Motion

In [25]:
def simulate_price_paths(
    initial_price: float,
    volatility: float,
    days: int,
    num_simulations: int,
    drift: float = 0
) -> np.ndarray:
    """
    Simulate future price paths using Geometric Brownian Motion
    """
    dt = 1/365  # Daily steps
    steps = days

    # Generate price paths
    returns = np.random.normal(
        (drift - 0.5 * volatility**2) * dt,
        volatility * np.sqrt(dt),
        size=(num_simulations, steps)
    )

    price_paths = initial_price * np.exp(np.cumsum(returns, axis=1))
    return price_paths

In [26]:
def run_simulation(
    initial_liquidity: float,
    initial_positions: List[Position],
    initial_prices: Dict[str, List[float]],  # Must contain 20 prices per asset
    volatility: Dict[str, float],
    simulation_days: int,
    num_simulations: int,
) -> Dict:
    """
    Run multiple simulations of the DeFi vault
    """
    results = {
        'liquidations_by_sim': [],
        'final_liquidity_by_sim': [],
        'lowest_liquidity_by_sim': []
    }

    # Run multiple simulations
    for sim in range(num_simulations):
        vault = DeFiVault(initial_liquidity)

        # initialize prices
        for asset, prices in initial_prices.items():
            vault.initialize_price_history(asset, prices)

        # Add initial positions
        for position in initial_positions:
            vault.add_position(position)

        # Generate price paths for each asset
        price_paths = {
            asset: simulate_price_paths(
                initial_price=prices[-1],  # Start from last known price
                volatility=volatility[asset],
                days=simulation_days,
                num_simulations=1
            )[0] for asset, prices in initial_prices.items()
        }

        liquidation_count = 0
        min_liquidity = initial_liquidity

        # Step through simulation
        for day in range(simulation_days):
            current_prices = {
                asset: paths[day] for asset, paths in price_paths.items()
            }

            liquidations = vault.check_liquidations(
                day,
            )
            liquidation_count += len(liquidations)
            min_liquidity = min(min_liquidity, vault.total_liquidity)

        results['liquidations_by_sim'].append(liquidation_count)
        results['final_liquidity_by_sim'].append(vault.total_liquidity)
        results['lowest_liquidity_by_sim'].append(min_liquidity)

        print(results)

    return results

# Example usage and analysis
def analyze_simulation_results(results: Dict) -> Dict:
    """Analyze and summarize simulation results"""
    analysis = {
        'avg_liquidations': np.mean(results['liquidations_by_sim']),
        'max_liquidations': np.max(results['liquidations_by_sim']),
        'liquidation_95th_percentile': np.percentile(results['liquidations_by_sim'], 95),
        'avg_final_liquidity': np.mean(results['final_liquidity_by_sim']),
        'min_final_liquidity': np.min(results['final_liquidity_by_sim']),
        'probability_of_insolvency': np.mean(
            [l <= 0 for l in results['lowest_liquidity_by_sim']]
        )
    }
    return analysis

In [27]:
import random

eth_initial_prices = [
    2518, 2480, 2430, 2519, 2520, 2530, 2600, 2500, 2488, 2450,
    2518, 2480, 2430, 2519, 2520, 2530, 2600, 2500, 2488, 2450]
usdc_initial_prices = [1] * 20

initial_prices = {
    "eth": eth_initial_prices,
    "usdc": usdc_initial_prices
}

# Get the latest prices
latest_eth_price = eth_initial_prices[-1]  # 2450
latest_usdc_price = usdc_initial_prices[-1]  # 1

# Generate positions with 90% LTV
initial_positions = []
for i in range(20):
    collateral_amount = random.uniform(1, 100)
    collateral_value = collateral_amount * latest_eth_price

    borrowed_amount = collateral_value * random.uniform(0, 0.8)

    position = Position(
        collateral_amount=collateral_amount,
        collateral_asset="eth",
        borrowed_amount=borrowed_amount,
        borrowed_asset="usdc",
    )

    initial_positions.append(position)

# Verify
for pos in initial_positions:
    collateral_value = pos.collateral_amount * latest_eth_price
    borrowed_value = pos.borrowed_amount * latest_usdc_price
    ltv = borrowed_value / collateral_value
    print(f"Position LTV: {ltv:.2%}")

volatility = {"eth": 0.50, "usdc": 0.01}

results = run_simulation(
    initial_liquidity=1000000,
    initial_positions=initial_positions,
    initial_prices=initial_prices,
    volatility=volatility,
    simulation_days=20,
    num_simulations=10,
)

analysis = analyze_simulation_results(results)

Position LTV: 18.76%
Position LTV: 58.18%
Position LTV: 68.19%
Position LTV: 42.10%
Position LTV: 13.22%
Position LTV: 11.84%
Position LTV: 37.61%
Position LTV: 61.51%
Position LTV: 76.70%
Position LTV: 1.79%
Position LTV: 38.25%
Position LTV: 54.40%
Position LTV: 54.27%
Position LTV: 51.61%
Position LTV: 61.82%
Position LTV: 14.33%
Position LTV: 63.92%
Position LTV: 55.09%
Position LTV: 15.62%
Position LTV: 4.22%
{'liquidations_by_sim': [0], 'final_liquidity_by_sim': [2431347.9957985887], 'lowest_liquidity_by_sim': [1000000]}
{'liquidations_by_sim': [0, 0], 'final_liquidity_by_sim': [2431347.9957985887, 2431347.9957985887], 'lowest_liquidity_by_sim': [1000000, 1000000]}
{'liquidations_by_sim': [0, 0, 0], 'final_liquidity_by_sim': [2431347.9957985887, 2431347.9957985887, 2431347.9957985887], 'lowest_liquidity_by_sim': [1000000, 1000000, 1000000]}
{'liquidations_by_sim': [0, 0, 0, 0], 'final_liquidity_by_sim': [2431347.9957985887, 2431347.9957985887, 2431347.9957985887, 2431347.99579858

KeyboardInterrupt: 